In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
dataa = pd.read_csv('bert_for_mvd.csv')
datab = pd.read_csv('sentiment_for_mvd.csv')
datac = pd.read_csv('lexical_for_mvd.csv')
datad = pd.read_csv('count_of_dicts_for_mvd.csv')

In [3]:
dataa = dataa.drop(columns = ['Unnamed: 0'])
datab = datab.drop(columns = ['Unnamed: 0'])
datac = datac.drop(columns = ['Unnamed: 0'])
datad = datad.drop(columns = ['Unnamed: 0'])

In [4]:
data = pd.read_csv('MVD_Dataset_Final.csv')
datae = data['LABEL']
result = pd.concat([dataa, datab, datac, datad, datae], axis=1)
result

,0,1,2,3,4,5,6,7,8,9,...,If a Question,Count of Emojis,Relative Freq of Caps,Count of Punctuation Marks,Count of Celebrity Mentions,Count of Slangs,Count of Porn Words,Count of Celebrity Related Words,Count of Clickbait Phrases,LABEL
0,0.656415,-0.588487,0.255769,-0.457658,0.070014,-0.430282,0.208616,0.499660,-0.069495,0.193156,...,0,1,0.271186,4,1,0,0,3,1,Clickbait
1,0.366475,-0.105111,-0.511447,-0.280992,0.036200,-0.419664,0.577570,0.173368,-0.598301,0.010184,...,0,1,0.133333,0,1,0,0,1,0,Clickbait
2,0.442341,-0.153139,0.180063,-0.506507,0.685487,-0.694880,0.302318,0.311490,-0.100956,0.340628,...,0,1,0.102941,1,1,0,0,0,0,Clickbait
3,0.189013,-0.487068,-0.017525,-0.428642,0.460981,-0.412667,0.287481,0.204468,-0.091526,0.142367,...,0,0,0.347826,0,0,0,0,0,0,Clickbait
4,0.309474,-0.757419,-0.338226,-0.159227,0.707320,-0.499518,0.264550,0.264980,0.061713,0.041802,...,0,0,0.129032,0,0,0,1,0,0,Clickbait
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,-0.188280,-0.076149,0.133796,-0.753641,0.576994,-0.105465,0.646150,0.314543,0.011651,0.218404,...,0,1,0.125000,0,1,0,0,0,0,Real
988,0.142554,-0.599347,0.047055,-0.120339,0.484493,-0.286151,-0.189977,0.274936,-0.436062,0.490000,...,0,1,0.212766,1,1,0,0,0,0,Real
989,0.230374,-0.681157,0.077860,-0.531160,0.515146,-0.507808,0.250623,0.199079,-0.494489,0.335102,...,0,1,0.171875,3,1,0,0,0,0,Real
990,-0.005640,-0.443048,-0.027411,-0.430937,0.478407,-0.239432,0.100175,0.254166,-0.252279,0.371059,...,0,1,0.214286,3,1,0,0,0,0,Real


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
result['LABEL'] = le.fit_transform(result['LABEL'])
data0 = result

In [8]:
# shuffling the rows in the dataset so that when splitting the train and test set are equally distributed
data0 = data0.sample(frac=1).reset_index(drop=True)
data = data0

In [9]:
# Sepratating & assigning features and target columns to X & y
y = data['LABEL']
X = data.drop(['LABEL'],axis=1)

In [10]:
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
fs = SelectKBest(score_func=f_classif, k=200)
X_selected=fs.fit_transform(X,y)

In [11]:
# Splitting the dataset into train and test sets: 80-20 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_selected, y, 
                                                    test_size = 0.2, random_state = 12)
X_train.shape, X_test.shape

((793, 200), (199, 200))

In [12]:
# Decision Tree model 
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth = 5)
#tree.fit(X_train, y_train)
#y_pred_dc = tree.predict(X_test)
clf = make_pipeline(preprocessing.StandardScaler(), tree)
scores = cross_val_score(clf,X_selected,y, cv = 5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.81 (+/- 0.07)


In [13]:
#XGBoost Classification model
from xgboost import XGBClassifier
xgb = XGBClassifier(learning_rate=0.4,max_depth=7)
#xgb.fit(X_train, y_train)
#y_pred_xgb = xgb.predict(X_test)
#print('XGB: ' + (str)(accuracy_score(y_test,y_pred_xgb)))
clf = make_pipeline(preprocessing.StandardScaler(), xgb)
scores = cross_val_score(clf,X_selected,y, cv = 5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.90 (+/- 0.04)
